In [72]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import datetime

In [28]:

# path0 = 'Q:\\SHA_KQ\\Digital Intelligence\\深度赋智-AI\\FTP\\hp_train\\2019 outbound\\'
# path1 = 'Q:\\SHA_KQ\\Digital Intelligence\\深度赋智-AI\\FTP\\hp_train\\2020 outbound\\'
# # path2 = 'Q:\\SHA_KQ\\Digital Intelligence\\深度赋智-AI\\FTP\\hp_train\\2021 outbound\\'
# path2 = 'Q:\\SHA_KQ\\Digital Intelligence\\深度赋智-AI\\FTP\\保乐力加_train\\outbound\\'
# %%time
# dfs0 = pd.DataFrame()
# for fname in os.listdir(path0):
#     if re.search(r'\.xls$', fname):
#         dfs0 = pd.concat(
#             [dfs0 ,pd.read_excel(path0 + fname)], 
#             axis = 0, 
#             ignore_index = True)

# dfs1 = pd.DataFrame()
# for fname in os.listdir(path1):
#     if re.search(r'\.xls$', fname):
#         dfs1 = pd.concat(
#             [dfs1 ,pd.read_excel(path1 + fname)], 
#             axis = 0, 
#             ignore_index = True)


# dfs2 = pd.DataFrame()
# for fname in os.listdir(path2):
#     if re.search(r'\.xls$', fname):
#         dfs2 = pd.concat(
#             [dfs2 ,pd.read_excel(path2 + fname)], 
#             axis = 0, 
#             ignore_index = True)
# dfs2 = dfs2.dropna(axis =1 , how = 'all')
 
# # path+
# dfs = pd.DataFrame()
# for fname in os.listdir(path):
#     if re.search(r'\.xls$', fname):
#         dfs = pd.concat(
#             [dfs ,pd.read_excel(path + fname)], axis = 0, ignore_index = True)
# prc_in = pd.DataFrame(dfs)
# prc_in = prc_in.drop(['仓库城市','仓库编号','Unnamed: 7'], axis = 1)
# prc_in.columns = ['time', 'sku','name','code','quant']

# dfs3 = pd.DataFrame()
# for fname in os.listdir(path2):
#     if re.search(r'\.xlsx$', fname):
#         dfs3 = pd.concat(
#             [dfs3 ,pd.read_excel(path3 + fname)], 
#             axis = 0, 
#             ignore_index = True)
# pd.set_option("display.max_rows", None, "display.max_columns", None)
  
# # dfs2.head(200)

In [71]:
# df = pd.read_clipboard()
# df.to_csv('./fas_057_rh.csv', index = False, encoding='utf_8_sig')
df = pd.read_csv('./fas_057_rh.csv')


In [76]:
df['ARRIVED_DATE_TIME'].str.slice(11,-1)

0       09:41:22.00
1       10:00:46.00
2       09:47:38.00
3       10:10:13.00
4       10:09:33.00
           ...     
8446    08:47:10.00
8447    08:45:59.00
8448    08:34:41.00
8449    08:50:06.00
8450    23:20:21.00
Name: ARRIVED_DATE_TIME, Length: 8451, dtype: object

In [79]:
def date_mutate(col):
    df[col + '_time'] = df[col].str.slice(11,-1)
    df[col] = df[col].str.slice(0,10)
    
    return df.head(1)

# df.columns[df.columns.str.contains(r'date|DATE')].to_numpy()
# df[df['receipt_id'].str.findall(r'(货)').apply(len) > 1]

for i in df.columns[df.columns.str.contains(r'date|DATE')].to_numpy():
    date_mutate(i)


In [119]:
first_df = df.groupby(
    'CREATION_DATE_TIME_STAMP'
    )['CREATION_DATE_TIME_STAMP_time'].min().reset_index()

first_df = first_df.merge(
    df, 
    on = ['CREATION_DATE_TIME_STAMP', 'CREATION_DATE_TIME_STAMP_time'], 
    how = 'left')

first_df2 = df.copy().groupby('CLOSE_DATE')[
    ['receipt_id', 'receipt_date','total_lines', 'TOTAL_QTY']
    ].sum().reset_index()

first_df2.columns = ['CLOSE_DATE', 'ttl_line_cnt', 'ttl_qty_sum']

first_df = first_df.merge(first_df2, how = 'inner', on = 'CLOSE_DATE')

In [138]:

first_df.groupby(
    'CLOSE_DATE'
    ).agg({'receipt_id':['nunique'], 'TOTAL_QTY': ['sum']}).reset_index()

,CLOSE_DATE,receipt_id,TOTAL_QTY
,,nunique,sum
0,2017-09-14,3,6884.0
1,2017-09-18,2,1344.0
2,2017-09-20,2,53815.0
3,2017-09-22,2,621.0
4,2017-09-25,1,6300.0
...,...,...,...
608,2021-04-27,1,24192.0
609,2021-04-28,1,1.0
610,2021-04-29,1,1008.0


---

In [114]:
ttl_qty_sum = first_df.pop('ttl_qty_sum')
first_df.pop('ttl_line_cnt')

In [118]:
first_df

,CREATION_DATE_TIME_STAMP,CREATION_DATE_TIME_STAMP_time,Internal_Receipt_num,warehouse,company,receipt_id,receipt_date,CLOSE_DATE,ARRIVED_DATE_TIME,total_lines,TOTAL_QTY,TOTAL_CONTAINERS,TOTAL_WEIGHT,receipt_date_time,CLOSE_DATE_time,ARRIVED_DATE_TIME_time
0,2017-09-11,03:24:56.00,12318,FAS,Solvay-SHANGHAI,ZHUANGUAN/4512966525,2017-09-11,2017-09-14,2017-09-14,2,880.0,22,22000.00,00:00:00.00,13:36:41.67,13:30:40.00
1,2017-09-13,01:22:11.00,12333,FAS,Solvay-SHANGHAI,205-82893856/320411889086,2017-09-13,2017-09-14,2017-09-14,1,4.0,1,100.00,00:00:00.00,13:03:11.09,13:02:51.00
2,2017-09-14,00:24:02.00,12342,FAS,Pernod,20170914 CHANGE CODE-1,2017-09-14,2017-09-14,2017-09-14,1,6000.0,9,8000.00,00:00:00.00,15:24:34.92,15:19:44.00
3,2017-09-12,05:07:37.00,12321,FAS,Solvay-SHANGHAI,ZHUANGUAN/4512969246,2017-09-12,2017-09-18,2017-09-18,1,744.0,19,18600.00,00:00:00.00,10:16:20.23,10:12:39.00
4,2017-09-18,00:58:48.00,12357,FAS,Wilson,822238760,2017-09-18,2017-09-18,2017-09-18,1,600.0,1,0.00,00:00:00.00,16:59:30.83,16:58:53.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,2021-04-25,07:59:55.00,22156,FAS,Pernod,EGLV540100041630,2021-04-25,2021-04-27,2021-04-27,3,24192.0,18,18486.72,00:00:00.00,05:59:32.16,05:48:50.00
889,2021-04-28,05:35:57.00,22162,FAS,Pernod,227-50319861 M2100737,2021-04-28,2021-04-29,2021-04-29,2,1008.0,3,1290.24,00:00:00.00,07:18:11.58,07:16:57.00
890,2021-04-29,05:10:23.00,22163,FAS,Pernod,COSU6288760580,2021-04-29,2021-05-06,2021-05-06,1,10080.0,10,12516.00,00:00:00.00,02:21:41.22,02:19:05.00
891,2021-04-30,04:52:53.00,22168,FAS,Pernod,ONEYLIVB05481700,2021-04-30,2021-05-07,2021-05-07,1,16632.0,1,17269.56,00:00:00.00,08:51:48.65,08:50:06.00


In [110]:
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn import model_selection
cv_split = model_selection.ShuffleSplit(
    n_splits = 6, 
    test_size = .25, 
    train_size = .75, 
    random_state = 1026529)
alg1 = Lasso()
cv_results_1 = model_selection.cross_validate(alg1, first_df, ttl_qty_sum, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_1['test_score'].mean()

In [112]:
from sklearn.ensemble import RandomForestRegressor
alg2 = RandomForestRegressor(n_estimators = 11, 
        max_depth = 31,
        min_samples_split = 3, 
        min_samples_leaf=2, 
        # max_leaf_nodes = 31
        )
cv_results_2 = model_selection.cross_validate(alg2, bs_train, bs_tar, cv  = cv_split, scoring='neg_mean_absolute_percentage_error')
cv_results_2['test_score'].mean()